# 环境与分布偏移
:label:`sec_environment-and-distribution-shift`

在前面的章节中，我们通过一系列机器学习的实际应用，将模型拟合到各种数据集上。然而，我们从未停下来思考数据最初从何而来，或者我们最终打算如何使用模型的输出。很多时候，拥有数据的机器学习开发者会急于开发模型，而没有停下来考虑这些基本问题。

许多失败的机器学习部署可以追溯到这种失败。有时，模型在测试集上的准确率看起来非常出色，但在部署时，当数据分布突然发生变化时，模型可能会灾难性地失败。更隐蔽的是，有时模型的部署本身可能是导致数据分布变化的催化剂。例如，假设我们训练了一个模型来预测谁会偿还贷款而不是违约，发现申请人的鞋类选择与违约风险相关（牛津鞋表示还款，运动鞋表示违约）。此后，我们可能会倾向于向穿牛津鞋的所有申请人发放贷款，并拒绝所有穿运动鞋的申请人。

在这种情况下，我们从模式识别到决策的草率跳跃，以及未能批判性地考虑环境，可能会带来灾难性的后果。首先，一旦我们开始根据鞋类做出决策，客户就会意识到并改变他们的行为。不久之后，所有申请人都会穿着牛津鞋，但信用状况并不会相应改善。花点时间消化这一点，因为在许多机器学习应用中都存在类似的问题：通过将基于模型的决策引入环境中，我们可能会破坏模型。

虽然我们不可能在一个章节中全面讨论这些话题，但我们旨在这里揭示一些常见的问题，并激发必要的批判性思维，以便及早发现此类情况，减轻损害，并负责任地使用机器学习。有些解决方案很简单（要求“正确”的数据），有些技术上很困难（实现强化学习系统），还有一些则需要我们走出统计预测的领域，去应对有关算法伦理应用的棘手哲学问题。

## 分布偏移的类型

首先，我们坚持被动预测设置，考虑数据分布可能以各种方式变化，以及可以采取什么措施来挽救模型性能。在一个经典设置中，我们假设我们的训练数据是从某个分布 $p_S(\mathbf{x},y)$ 中采样的，但我们的测试数据将由来自不同分布 $p_T(\mathbf{x},y)$ 的未标记样本组成。已经，我们必须面对一个严峻的现实。在没有任何关于 $p_S$ 和 $p_T$ 之间关系的假设的情况下，学习一个稳健的分类器是不可能的。

考虑一个二元分类问题，其中我们希望区分狗和猫。如果分布可以任意变化，那么我们的设置允许出现这样的病态情况：输入的分布保持不变：$p_S(\mathbf{x}) = p_T(\mathbf{x})$，但标签全部反转：$p_S(y \mid \mathbf{x}) = 1 - p_T(y \mid \mathbf{x})$。换句话说，如果上帝突然决定将来所有的“猫”现在都是狗，而我们以前称之为“狗”的现在是猫——而没有任何输入分布 $p(\mathbf{x})$ 的变化，那么我们就无法将这种情况与分布根本没有变化的情况区分开来。

幸运的是，在对未来数据可能变化的一些限制性假设下，有原则的算法可以检测偏移，有时甚至可以实时适应，提高原始分类器的准确性。

### 协变量偏移

在分布偏移类别中，协变量偏移可能是研究最广泛的。在这里，我们假设虽然输入的分布在随时间变化，但标签函数，即条件分布 $P(y \mid \mathbf{x})$ 不变。统计学家称之为*协变量偏移*，因为问题是由于协变量（特征）的分布变化引起的。虽然我们有时可以在不涉及因果关系的情况下推理分布偏移，但在我们认为 $\mathbf{x}$ 导致 $y$ 的情况下，协变量偏移是自然的假设。

考虑区分猫和狗的挑战。我们的训练数据可能包含 :numref:`fig_cat-dog-train` 中所示类型的图像。

![用于区分猫和狗的训练数据（插图：Lafeez Hossain / 500px / Getty Images; ilkermetinkursova / iStock / Getty Images Plus; GlobalP / iStock / Getty Images Plus; Musthafa Aboobakuru / 500px / Getty Images）。](../img/cat-dog-train.png)
:label:`fig_cat-dog-train`


在测试时，我们被要求对 :numref:`fig_cat-dog-test` 中的图像进行分类。

![用于区分猫和狗的测试数据（插图：SIBAS_minich / iStock / Getty Images Plus; Ghrzuzudu / iStock / Getty Images Plus; id-work / DigitalVision Vectors / Getty Images; Yime / iStock / Getty Images Plus）。](../img/cat-dog-test.png)
:label:`fig_cat-dog-test`

训练集由照片组成，而测试集仅包含卡通。在没有连贯计划适应新领域的情况下，用具有显著不同特征的数据集进行训练可能会带来麻烦。

### 标签偏移

*标签偏移* 描述了相反的问题。在这里，我们假设标签边缘 $P(y)$ 可以变化，但类条件分布 $P(\mathbf{x} \mid y)$ 在不同领域中保持固定。当我们认为 $y$ 导致 $\mathbf{x}$ 时，标签偏移是一个合理的假设。例如，我们可能希望根据症状（或其他表现）预测诊断，即使诊断的相对流行度随着时间的推移而变化。标签偏移在这里是适当的假设，因为疾病会导致症状。在某些退化情况下，标签偏移和协变量偏移假设可以同时成立。例如，当标签是确定性的时，即使 $y$ 导致 $\mathbf{x}$，协变量偏移假设也将得到满足。有趣的是，在这些情况下，通常有利的是采用源自标签偏移假设的方法。这是因为这些方法往往涉及操作看起来像标签的对象（通常是低维的），而不是看起来像输入的对象，后者在深度学习中往往是高维的。

### 概念偏移

我们还可能遇到相关的*概念偏移*问题，当标签的定义可以随时间变化时会出现这种情况。这听起来很奇怪——一只*猫*就是一只*猫*，不是吗？然而，其他类别会随着时间的推移在使用上发生变化。精神疾病的诊断标准、时尚的东西以及职位头衔都经历了大量的概念偏移。事实证明，如果我们围绕美国各地移动，按地理来源转移我们的数据，我们会发现关于*软饮料*名称的分布存在相当大的概念偏移，如 :numref:`fig_popvssoda` 所示。

![美国软饮料名称的概念偏移（CC-BY: Alan McConchie, PopVsSoda.com）。](../img/popvssoda.png)
:width:`400px`
:label:`fig_popvssoda`

如果我们构建一个机器翻译系统，分布 $P(y \mid \mathbf{x})$ 可能会因我们的位置而异。这个问题可能很难发现。我们可能希望利用知识，即偏移只会在时间或地理意义上逐渐发生。

## 分布偏移的例子

在深入探讨形式主义和算法之前，我们可以讨论一些具体的协变量或概念偏移可能不明显的情况。

### 医学诊断

想象一下，你想设计一种癌症检测算法。你收集健康人和病人的数据并训练你的算法。它运行良好，给你很高的准确性，你得出结论，你已经准备好从事成功的医学诊断事业。*没这么快。*

产生训练数据的分布和你在野外遇到的分布可能相差很大。几年前，一些作者曾为一家不幸的初创公司工作过，这件事就发生在他们身上。他们正在开发一种血液测试来检测一种主要影响老年男性的疾病，并希望使用他们收集的患者血液样本来研究它。然而，获取健康男性血液样本比获取已经在系统中的病人的血液样本要困难得多。为了弥补这一点，这家初创公司请求大学校园的学生捐赠血液，作为开发测试时的健康对照组。然后他们问我们是否可以帮助他们建立一个检测疾病的分类器。

正如我们向他们解释的那样，确实很容易以接近完美的准确性区分健康和患病队列。然而，这是因为测试对象在年龄、激素水平、身体活动、饮食、酒精消费等许多与疾病无关的因素上存在差异。真正的病人不太可能出现这种情况。由于他们的抽样程序，我们可以预期会遇到极端的协变量偏移。此外，这种情况不太可能通过传统方法纠正。简而言之，他们浪费了一大笔钱。

### 自动驾驶汽车

假设一家公司想利用机器学习开发自动驾驶汽车。其中一个关键组件是路边探测器。由于真实的标注数据非常昂贵，他们有了一个（聪明且有问题的）想法，即使用游戏渲染引擎中的合成数据作为额外的训练数据。这在从渲染引擎中抽取的“测试数据”上效果非常好。然而，在真正的汽车里却是一场灾难。原来，路边是用非常简单的纹理渲染的。更重要的是，*所有*路边都用*相同的*纹理渲染，路边探测器很快就学会了这个“特征”。

类似的事情也发生在美军第一次尝试在森林中探测坦克时。他们拍摄了没有坦克的森林的航拍照片，然后把坦克开进森林，再拍摄另一组照片。分类器似乎*完美*地工作了。不幸的是，它只是学会了如何区分有阴影的树和没有阴影的树——第一组照片是在清晨拍摄的，第二组照片是在中午拍摄的。

### 非平稳分布

当分布缓慢变化（也称为*非平稳分布*）并且模型没有充分更新时，会出现更为微妙的情况。以下是一些典型情况。

* 我们训练了一个计算广告模型，但没有频繁更新（例如，我们忘记纳入一款名为iPad的新设备刚刚推出）。
* 我们构建了一个垃圾邮件过滤器。它很好地检测了迄今为止看到的所有垃圾邮件。但随后垃圾邮件发送者变得聪明起来，制作了看起来与我们以前见过的任何东西都不一样的新消息。
* 我们构建了一个产品推荐系统。它在整个冬天都有效，但圣诞节过后仍然继续推荐圣诞帽。

### 更多轶事

* 我们构建了一个面部检测器。它在所有基准测试中表现良好。不幸的是，它在测试数据上失败了——有问题的例子是特写镜头，整个图像中充满了脸（训练集中没有这样的数据）。
* 我们为美国市场构建了一个网络搜索引擎，并希望将其部署到英国。
* 我们通过编译一个大型数据集来训练一个图像分类器，其中大量类别在数据集中均等地表示，比如说1000个类别，每个类别由1000张图像表示。然后我们在实际世界中部署该系统，其中实际的照片标签分布显然是非均匀的。

## 分布偏移的修正

正如我们所讨论的，有许多情况下训练和测试分布 $P(\mathbf{x}, y)$ 是不同的。在某些情况下，我们很幸运，尽管存在协变量、标签或概念偏移，模型仍然有效。在其他情况下，我们可以通过采用有原则的策略来应对偏移，从而做得更好。本节其余部分的技术性大大增强。不耐烦的读者可以继续阅读下一节，因为这部分内容并不是后续概念的先决条件。

### 经验风险与风险
:label:`subsec_empirical-risk-and-risk`

让我们首先反思一下在模型训练过程中到底发生了什么：我们迭代遍历训练数据 $\{(\mathbf{x}_1, y_1), \ldots, (\mathbf{x}_n, y_n)\}$ 的特征和关联标签，并在每个小批量后更新模型 $f$ 的参数。为简单起见，我们不考虑正则化，因此我们主要最小化训练损失：

$$\mathop{\mathrm{minimize}}_f \frac{1}{n} \sum_{i=1}^n l(f(\mathbf{x}_i), y_i),$$
:eqlabel:`eq_empirical-risk-min`

其中 $l$ 是衡量给定关联标签 $y_i$ 时预测 $f(\mathbf{x}_i)$ 的“糟糕程度”的损失函数。统计学家称 :eqref:`eq_empirical-risk-min` 中的项为*经验风险*。*经验风险*是训练数据上平均损失，用于近似*风险*，即从其真实分布 $p(\mathbf{x},y)$ 中抽取的整个数据总体的损失期望值：

$$E_{p(\mathbf{x}, y)} [l(f(\mathbf{x}), y)] = \int\int l(f(\mathbf{x}), y) p(\mathbf{x}, y) \;d\mathbf{x}dy.$$
:eqlabel:`eq_true-risk`

然而，在实践中，我们通常无法获得整个数据总体。因此，*经验风险最小化*，即最小化 :eqref:`eq_empirical-risk-min` 中的经验风险，是一种实用的机器学习策略，希望大约最小化风险。

### 协变量偏移修正
:label:`subsec_covariate-shift-correction`

假设我们想要估计某种依赖关系 $P(y \mid \mathbf{x})$，对此我们有带标签的数据 $(\mathbf{x}_i, y_i)$。不幸的是，观察 $\mathbf{x}_i$ 是从某个*源分布* $q(\mathbf{x})$ 而不是*目标分布* $p(\mathbf{x})$ 抽取的。幸运的是，依赖假设意味着条件分布不会改变：$p(y \mid \mathbf{x}) = q(y \mid \mathbf{x})$。如果源分布 $q(\mathbf{x})$ 是“错误的”，我们可以通过使用风险中的以下简单恒等式来修正：

$$
\begin{aligned}
\int\int l(f(\mathbf{x}), y) p(y \mid \mathbf{x})p(\mathbf{x}) \;d\mathbf{x}dy =
\int\int l(f(\mathbf{x}), y) q(y \mid \mathbf{x})q(\mathbf{x})\frac{p(\mathbf{x})}{q(\mathbf{x})} \;d\mathbf{x}dy.
\end{aligned}
$$

换句话说，我们需要按照正确的分布抽取的概率与错误的分布抽取的概率之比重新加权每个数据实例：

$$\beta_i \stackrel{\textrm{def}}{=} \frac{p(\mathbf{x}_i)}{q(\mathbf{x}_i)}.$$

插入每个数据实例 $(\mathbf{x}_i, y_i)$ 的权重 $\beta_i$，我们可以使用*加权经验风险最小化*来训练我们的模型：

$$\mathop{\mathrm{minimize}}_f \frac{1}{n} \sum_{i=1}^n \beta_i l(f(\mathbf{x}_i), y_i).$$
:eqlabel:`eq_weighted-empirical-risk-min`

唉，我们不知道这个比率，所以在做任何有用的事情之前，我们需要估计它。有许多方法可用，包括一些试图直接重新校准期望算子的高级算子理论方法，这些方法使用最小范数或最大熵原理。请注意，对于任何这样的方法，我们需要从两个分布中抽取的样本——“真实”的 $p$（例如，通过访问测试数据）和用于生成训练集的 $q$（后者显然可用）。但是请注意，我们只需要特征 $\mathbf{x} \sim p(\mathbf{x})$；我们不需要访问标签 $y \sim p(y)$。

在这种情况下，存在一种非常有效的方法，几乎可以达到与原始方法相同的效果：即逻辑回归，这是softmax回归（见 :numref:`sec_softmax`）的特殊情况，用于二分类。这就是计算估计概率比率所需的全部。我们学习一个分类器来区分从 $p(\mathbf{x})$ 抽取的数据和从 $q(\mathbf{x})$ 抽取的数据。如果无法区分这两个分布，则意味着相关实例同样可能来自这两个分布中的任何一个。另一方面，任何可以很好区分的实例都应该相应地显著增加权重或减少权重。

为简单起见，假设我们从两个分布 $p(\mathbf{x})$ 和 $q(\mathbf{x})$ 中分别有相等数量的实例。现在用 $z$ 表示从 $p$ 抽取的数据的标签为1，从 $q$ 抽取的数据的标签为-1。那么混合数据集中的概率由

$$P(z=1 \mid \mathbf{x}) = \frac{p(\mathbf{x})}{p(\mathbf{x})+q(\mathbf{x})} \text{ 因此 } \frac{P(z=1 \mid \mathbf{x})}{P(z=-1 \mid \mathbf{x})} = \frac{p(\mathbf{x})}{q(\mathbf{x})}.$$

因此，如果我们采用逻辑回归方法，其中 $P(z=1 \mid \mathbf{x})=\frac{1}{1+\exp(-h(\mathbf{x}))}$（$h$ 是参数化的函数），则

$$
\beta_i = \frac{1/(1 + \exp(-h(\mathbf{x}_i)))}{\exp(-h(\mathbf{x}_i))/(1 + \exp(-h(\mathbf{x}_i)))} = \exp(h(\mathbf{x}_i)).
$$

因此，我们需要解决两个问题：第一个是区分从两个分布中抽取的数据，然后是 :eqref:`eq_weighted-empirical-risk-min` 中的加权经验风险最小化问题，其中我们按 $\beta_i$ 对项进行加权。

现在我们准备描述一个修正算法。假设我们有一个训练集 $\{(\mathbf{x}_1, y_1), \ldots, (\mathbf{x}_n, y_n)\}$ 和一个无标签的测试集 $\{\mathbf{u}_1, \ldots, \mathbf{u}_m\}$。对于协变量偏移，我们假设所有 $1 \leq i \leq n$ 的 $\mathbf{x}_i$ 都是从某个源分布中抽取的，而所有 $1 \leq i \leq m$ 的 $\mathbf{u}_i$ 都是从目标分布中抽取的。以下是修正协变量偏移的原型算法：

1. 创建一个二分类训练集：$\{(\mathbf{x}_1, -1), \ldots, (\mathbf{x}_n, -1), (\mathbf{u}_1, 1), \ldots, (\mathbf{u}_m, 1)\}$。
1. 使用逻辑回归训练一个二分类器以获得函数 $h$。
1. 使用 $\beta_i = \exp(h(\mathbf{x}_i))$ 或更好的 $\beta_i = \min(\exp(h(\mathbf{x}_i)), c)$ 对训练数据进行加权，其中 $c$ 是某个常数。
1. 在 :eqref:`eq_weighted-empirical-risk-min` 中使用权重 $\beta_i$ 训练 $\{(\mathbf{x}_1, y_1), \ldots, (\mathbf{x}_n, y_n)\}$。

请注意，上述算法依赖于一个关键假设。为了让该方案工作，我们需要目标（例如，测试时）分布中的每个数据实例在训练时都有非零的概率出现。如果我们找到一个点，其中 $p(\mathbf{x}) > 0$ 但 $q(\mathbf{x}) = 0$，那么相应的权重应该为无穷大。

### 标签偏移修正

假设我们正在处理一个具有 $k$ 类别的分类任务。使用 :numref:`subsec_covariate-shift-correction` 中的相同符号，$q$ 和 $p$ 分别是源分布（例如，训练时）和目标分布（例如，测试时）。假设标签的分布随时间变化：$q(y) \neq p(y)$，但类条件分布保持不变：$q(\mathbf{x} \mid y)=p(\mathbf{x} \mid y)$。如果源分布 $q(y)$ 是“错误的”，我们可以根据 :eqref:`eq_true-risk` 中定义的风险中的以下恒等式来修正：

$$
\begin{aligned}
\int\int l(f(\mathbf{x}), y) p(\mathbf{x} \mid y)p(y) \;d\mathbf{x}dy =
\int\int l(f(\mathbf{x}), y) q(\mathbf{x} \mid y)q(y)\frac{p(y)}{q(y)} \;d\mathbf{x}dy.
\end{aligned}
$$

这里的权重将对应于标签似然比：

$$\beta_i \stackrel{\textrm{def}}{=} \frac{p(y_i)}{q(y_i)}.$$

标签偏移的一个好处是，如果我们有一个在源分布上相当好的模型，那么我们可以一致地估计这些权重，而无需处理环境维度。在深度学习中，输入往往是高维对象，如图像，而标签往往是更简单的对象，如类别。

为了估计目标标签分布，我们首先采用一个相当好的现成分类器（通常在训练数据上训练），并使用验证集（也来自训练分布）计算其“混淆矩阵”。*混淆矩阵* $\mathbf{C}$ 只是一个 $k \times k$ 矩阵，其中每一列对应于标签类别（真实标签），每一行对应于我们模型预测的类别。每个单元格的值 $c_{ij}$ 是验证集中真实标签为 $j$ 且我们模型预测为 $i$ 的总预测分数。

现在，我们不能直接在目标数据上计算混淆矩阵，因为我们看不到野外看到的示例的标签，除非我们投资于复杂的实时注释管道。不过，我们可以做的，是对测试时我们模型的所有预测求平均，得到平均模型输出 $\mu(\hat{\mathbf{y}}) \in \mathbb{R}^k$，其中第 $i$ 个元素 $\mu(\hat{y}_i)$ 是测试集中我们模型预测为 $i$ 的总预测分数。

事实证明，在一些温和的条件下——如果我们的分类器本来就足够准确，并且目标数据只包含我们之前见过的类别，并且标签偏移假设本身就成立（这里最强的假设）——我们可以通过解一个简单的线性系统来估计测试集标签分布

$$\mathbf{C} p(\mathbf{y}) = \mu(\hat{\mathbf{y}}),$$

因为作为一个估计 $\sum_{j=1}^k c_{ij} p(y_j) = \mu(\hat{y}_i)$ 对所有 $1 \leq i \leq k$ 成立，其中 $p(y_j)$ 是 $k$ 维标签分布向量 $p(\mathbf{y})$ 的第 $j$ 个元素。如果我们的分类器一开始就足够准确，那么混淆矩阵 $\mathbf{C}$ 将是可逆的，我们得到解 $p(\mathbf{y}) = \mathbf{C}^{-1} \mu(\hat{\mathbf{y}})$。

因为我们观察到了源数据上的标签，所以很容易估计分布 $q(y)$。然后，对于带有标签 $y_i$ 的任何训练示例 $i$，我们可以取估计的 $p(y_i)/q(y_i)$ 的比率来计算权重 $\beta_i$，并将其插入 :eqref:`eq_weighted-empirical-risk-min` 中的加权经验风险最小化。

### 概念偏移修正

概念偏移在原则上更难修复。例如，在突然从区分猫和狗变为区分白色和黑色动物的情况下，假设我们可以做得比收集新标签并从头开始训练更好是不合理的。幸运的是，在实践中，这种极端的偏移很少见。相反，通常会发生的是任务慢慢变化。为了让事情更具体，这里有一些例子：

* 在计算广告中，新产品推出，旧产品变得不那么受欢迎。这意味着广告及其受欢迎程度的分布逐渐变化，任何点击率预测器都需要随之逐渐变化。
* 交通摄像头镜头由于环境磨损而逐渐退化，逐步影响图像质量。
* 新闻内容逐渐变化（即，大多数新闻保持不变，但新故事出现）。

在这些情况下，我们可以使用与训练网络使其适应数据变化相同的方法。换句话说，我们使用现有的网络权重，只需用新数据进行几次更新步骤，而不是从头开始训练。

## 学习问题的分类

掌握了如何处理分布变化的知识后，我们现在可以考虑机器学习问题表述的其他方面。

### 批量学习

在*批量学习*中，我们有权访问训练特征和标签 $\{(\mathbf{x}_1, y_1), \ldots, (\mathbf{x}_n, y_n)\}$，我们使用这些数据训练模型 $f(\mathbf{x})$。稍后，我们将该模型部署以评分从同一分布中抽取的新数据 $(\mathbf{x}, y)$。这是我们在此讨论的任何问题的默认假设。例如，我们可能会基于大量猫和狗的图片训练一个猫检测器。一旦训练完成，我们会将其作为智能猫门计算机视觉系统的一部分发货，该系统只允许猫进入。然后安装在客户的家中，除非在极端情况下，否则不会再更新。

### 在线学习

现在想象数据 $(\mathbf{x}_i, y_i)$ 一次一个样本地到达。更具体地说，假设我们首先观察到 $\mathbf{x}_i$，然后我们需要提出一个估计 $f(\mathbf{x}_i)$。只有这样，我们才能观察到 $y_i$ 并因此获得奖励或遭受损失，基于我们的决策。许多实际问题属于这一类。例如，我们需要预测明天的股票价格，这使我们能够根据该估计进行交易，到一天结束时，我们发现我们的估计是否获利。换句话说，在*在线学习*中，我们不断改进模型，根据新的观察结果进行如下循环：

$$\begin{aligned}&\textrm{模型 } f_t \longrightarrow \textrm{数据 }  \mathbf{x}_t \longrightarrow \textrm{估计 } f_t(\mathbf{x}_t) \longrightarrow\\ \textrm{观}&\textrm{察 } y_t \longrightarrow \textrm{损失 } l(y_t, f_t(\mathbf{x}_t)) \longrightarrow \textrm{模型 } f_{t+1}\end{aligned}$$

### 强盗问题

*强盗问题*是上述问题的一个特例。虽然在大多数学习问题中，我们有一个连续参数化的函数 $f$，我们希望学习其参数（例如，一个深层网络），但在*强盗问题*中，我们只有有限数量的手臂可以拉动，即我们可以采取的有限数量的动作。对于这个更简单的问题，可以获得更强的最优性理论保证并不令人惊讶。我们主要列出它，因为这个问题经常（令人困惑地）被视为一种不同的学习设置。

### 控制

在许多情况下，环境会记住我们做了什么。不一定是以对抗的方式，但它会记住，反应将取决于之前发生的事情。例如，咖啡锅炉控制器会根据之前是否加热锅炉而观察到不同的温度。PID（比例-积分-微分）控制器算法是这类问题的流行选择。同样，用户在新闻网站上的行为将取决于我们之前向他们展示的内容（例如，他们只会阅读大部分新闻一次）。许多此类算法会为其所处的环境建模，以使其决策显得不那么随机。最近，控制理论（例如PID变体）也被用于自动调整超参数，以实现更好的解缠和重建质量，并提高生成文本的多样性和生成图像的重建质量 :cite:`Shao.Yao.Sun.ea.2020`。

### 强化学习

在记忆环境的一般情况下，我们可能会遇到环境试图与我们合作（合作游戏，特别是非零和游戏）的情况，或者环境会试图获胜的情况。国际象棋、围棋、西洋双陆棋或星际争霸是*强化学习*的一些案例。同样，我们可能希望为自动驾驶汽车建立一个好的控制器。其他车辆可能会以复杂的方式响应自动驾驶汽车的驾驶风格，例如试图避开它、试图造成事故或试图与其合作。

### 考虑环境

上述不同情况之间的一个关键区别在于，在静态环境中始终有效的策略，在可以适应的环境中可能不会始终有效。例如，交易员发现的套利机会很可能在被利用后消失。环境变化的速度和方式在很大程度上决定了我们可以使用的算法类型。例如，如果我们知道事物只能缓慢变化，我们可以强制任何估计也缓慢变化。如果我们知道环境可能会瞬间变化，但非常罕见，我们可以为此做好准备。对于有志成为数据科学家的人来说，这些类型的知识对于处理概念偏移（即，正在解决的问题会随时间变化）至关重要。

## 机器学习中的公平性、问责制和透明度

最后，重要的是要记住，当你部署机器学习系统时，你不仅仅是优化一个预测模型——你通常提供一个工具，将（部分或完全）自动化决策。这些技术系统可以影响受决策影响的个人的生活。从考虑预测到做出决策的飞跃不仅提出了新的技术问题，而且还提出了一系列必须仔细考虑的伦理问题。如果我们部署的是医疗诊断系统，我们需要知道它适用于哪些人群，哪些人群不适用。忽视可预见的对某一亚群体福利的影响可能导致我们提供劣质护理。此外，一旦我们考虑到决策系统，就必须退一步重新考虑如何评估我们的技术。这种范围的变化带来的众多后果之一是，*准确性* 很少是正确的衡量标准。例如，当将预测转化为行动时，我们通常希望考虑到以不同方式出错的成本敏感性。如果误分类图像的一种方式可能被视为种族歧视，而误分类到另一个类别则是无害的，那么我们可能希望相应地调整阈值，考虑到社会价值观来设计决策协议。我们还需要小心预测系统如何导致反馈循环。例如，考虑预测警务系统，它们将巡逻警力分配到犯罪率高的区域。很容易看出一个令人担忧的模式会出现：

 1. 犯罪更多的社区会得到更多的巡逻。
 1. 因此，在这些社区中发现更多犯罪，进入未来迭代的训练数据。
 1. 由于暴露于更多阳性样本，模型预测这些社区会有更多犯罪。
 1. 在下一次迭代中，更新后的模型更加关注同一个社区，导致发现更多犯罪，等等。

通常，在建模过程中，模型预测与训练数据耦合的各种机制都没有得到考虑。这可能会导致研究人员所说的*失控反馈循环*。此外，我们还需要小心是否一开始就在解决正确的问题。预测算法现在在信息传播中介中扮演着超乎寻常的角色。一个人遇到的新闻是否应该由他们在 Facebook 上点赞的页面集决定？这只是你在机器学习职业生涯中可能遇到的许多紧迫伦理困境中的几个。

## 总结

在许多情况下，训练集和测试集并不来自相同的分布。这称为分布偏移。风险是整个数据总体的损失期望值，这些数据是从其真实分布中抽取的。然而，整个数据总体通常不可用。经验风险是训练数据上的平均损失，用于近似风险。在实践中，我们执行经验风险最小化。

在相应的假设下，协变量和标签偏移可以在测试时被检测和修正。如果不考虑这种偏差，在测试时可能会出现问题。
在某些情况下，环境可能会记住自动化的行为并以意想不到的方式作出反应。在构建模型时，我们必须考虑到这种可能性，并继续监控实时系统，警惕我们的模型和环境可能会以意想不到的方式纠缠在一起。

## 练习

1. 当我们改变搜索引擎的行为时，会发生什么？用户会怎么做？广告商呢？
1. 实现一个协变量偏移检测器。提示：构建一个分类器。
1. 实现一个协变量偏移修正器。
1. 除了分布偏移之外，还有什么会影响经验风险对风险的近似？

[讨论](https://discuss.d2l.ai/t/105)